In [137]:
import collections
import math
import random

### Get a sieve of all numbers $n$ to indicate whether they are prime, with $0 \leq n \leq 10^7$

In [138]:
def get_primes_eratosthenes_sieve(n):
    is_prime = [1] * (n + 1)
    p = 2
    is_prime[0] = 0
    is_prime[1] = 0
    while p * p <= n:
        if is_prime[p]:
            for i in range(p * 2, n + 1, p):
                is_prime[i] = 0
        p += 1
    prime_list = []
    for i in range(n + 1):
        if is_prime[i]:
            prime_list.append(i)
    return is_prime

is_prime = get_primes_eratosthenes_sieve(10 ** 7)

### Use Pollard's Rho algorithm for finding prime factorizations for any $n$

In [139]:
# https://github.com/zhangbo2008/python_algorithm2/blob/c53669703b957a079f100c12711f86f5fc2f9389/algorithms/factorization/pollard_rho.py
def pollard_rho_prime_factorization(x):
    def f(x):
        return x * x + 1

    def rho(n, x1=2, x2=2):
        if n % 2 == 0:
            return 2
        i = 0
        while True:
            x1 = f(x1) % n
            x2 = f(f(x2)) % n
            divisor = math.gcd(abs(x1 - x2), n)
            i += 1
            if divisor != 1:
                break
            if i > 500:
                x1 = random.randint(1, 10)
                x2 = random.randint(1, 10)
                i = 0
        return divisor

    def pollard_rho_rec(x, factors):
        if x == 1:
            return

        if is_prime[x]:
            factors.append(x)
            return

        divisor = rho(x, random.randint(1, 10), random.randint(1, 10))
        pollard_rho_rec(divisor, factors)
        pollard_rho_rec(x // divisor, factors)

    if x == 1 or x == 0:
        return {x: 1}
    factors = []
    pollard_rho_rec(x, factors)
    return collections.Counter(factors)

### Find multiplicative inverse $x$ satisfying $xa \equiv 1\,(\text{mod b})$ and $\text{gcd}(a, b) = 1$

In [140]:
def mul_inv(a, b):
    if b == 1:
        return 1
    b_0 = b
    x_0, x_1 = 0, 1
    while a > 1:
        x_0, x_1 = x_1 - (a // b) * x_0, x_0
        a, b = b, a % b
    if x_1 < 0:
        x_1 += b_0
    return x_1

### Store prime factorizations for each $n$, with $1\leq n\leq N$ for rapid access 

In [141]:
N, M = 20000, 10 ** 9 + 7
prime_factorization = {n: pollard_rho_prime_factorization(n) for n in range(1, N + 1)}

### Compute prime factorization of $B(n)=\prod_{k=0}^n {n \choose k}$ by iteratively computing the prime factorizations of ${n\choose r} = \frac{n-r+1}{r}{n\choose r-1}$ and combining them multiplicatively

In [142]:
def B_prime_factorization(n):
    B_pf, binomial_pf = dict(), dict()
    for r in range(1, n // 2 + 1):
        # (n, r + 1) = (n, r) * (n - r) / (r + 1)
        multiply_pf, divide_pf = prime_factorization[n - r + 1], prime_factorization[r]
        for p in set(list(multiply_pf.keys()) + list(divide_pf.keys()) + list(binomial_pf.keys())):
            if p == 1:
                continue
            if p not in B_pf:
                B_pf[p] = 0
            if p not in binomial_pf:
                binomial_pf[p] = 0
        
            binomial_pf[p] += (multiply_pf[p] if p in multiply_pf else 0) - (divide_pf[p] if p in divide_pf else 0)
            B_pf[p] += (1 if r == n / 2 else 2) * binomial_pf[p]  # r == n / 2 only possible if n % 2 == 0
    
    return B_pf

### Use modular arithmetic to compute $D(k)=\sum_{d\mid B(n)}d=\prod_{k=0}^{K}\frac{p_k^{\alpha_k+1} - 1}{p_k-1}$, where $B(n)=\prod_{k=0}^K p_k^{\alpha_k}$ is the prime factorization of $B(n)$

In [134]:
multiplicative_inverses_p_1 = {p: mul_inv(p - 1, M) for p in [n for n in range(len(is_prime)) if is_prime[n] == 1]}

def D(n):
    B_pf, sum_of_divisors = B_prime_factorization(n), 1
    for p in B_pf:
        sum_of_divisors = (sum_of_divisors * (pow(p, B_pf[p] + 1, M) - 1) * multiplicative_inverses_p_1[p]) % M
        
    return sum_of_divisors

def S(n, M):
    summation_sum_of_divisors = 1  # Since S(1) = D(1) = 1 as B(1) = 1 * 1 = 1
    for k in range(2, n + 1):
        summation_sum_of_divisors += D(k) % M
        
        # Display progress of function execution for large values of N
        if k % 200 == 0:
            print('Progress:', k // 200, '%', end='\r', flush=True)
        
    return summation_sum_of_divisors % M

In [136]:
S(N, M)

538319652